# Importing Libraries

In [1]:
import sys
import pyrootutils

root = pyrootutils.setup_root(sys.path[0], pythonpath=True, cwd=True)

import timm
import torch
import shutil
import numpy as np
import torchvision
import seaborn as sns
import torch.nn as nn
import albumentations as A
import torch.optim as optim
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision.transforms as transforms


from PIL import Image
from omegaconf import OmegaConf
from torchvision import datasets
from hydra import compose, initialize
from torch import nn, optim, utils, Tensor
from albumentations.pytorch import ToTensorV2
from torch.utils.data import random_split, DataLoader, TensorDataset


shutil.copy("configs/config.yaml", "notebooks/config.yaml")
with initialize(version_base=None, config_path=""):
    config = compose(config_name="config.yaml")


/home/shivam13juna/Documents/virtual_envs/mlo/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
labels = {0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'} 


# Steps:
1. Make Data Loader
2. Make CIFAR 10 pytorch lightning Model (with both timm and custom)
3. Train and predict on hard-disk saved images



## Data Loader

In [3]:


cifar_mean = [0.49139968, 0.48215841, 0.44653091]
cifar_std =  [0.24703223, 0.24348513, 0.26158784]


class CIFARDataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str = 'data/', batch_size:int = 10000):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform = transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor(), transforms.Normalize(cifar_mean, cifar_std)])
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        
    def prepare_data(self):
        datasets.CIFAR10(self.data_dir, train=True, download=True)
        datasets.CIFAR10(self.data_dir, train=False, download=True)
    
    def setup(self, stage):

        self.train_full_set = datasets.CIFAR10(self.data_dir, train=True, download=True, transform=self.transform)

        self.cifar_train, self.cifar_val = random_split(self.train_full_set, [45000, 5000])

        self.cifar_test = datasets.CIFAR10(self.data_dir, train=False, download=True, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.cifar_train, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.cifar_val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.cifar_test, batch_size=self.batch_size)

    def predict_dataloader(self):
        return DataLoader(self.cifar_test, batch_size=self.batch_size)
 
         


        

class CIFARTrainingModule(pl.LightningModule):

    def __init__(self, units = 64):
        super().__init__()

        self.layer = nn.Sequential(nn.Linear(32 * 32 * 3, 512), nn.ReLU(), nn.Linear(512, 256), nn.ReLU(), nn.Linear(256, 128), nn.ReLU(), nn.Linear(128, 64),  nn.Linear(64, 10), nn.LogSoftmax(dim=-1))
    def training_step(self, batch, batch_idx):
        x, y = batch
        # print("Gotten batch and shape of x: ", x.shape)

        x = x.view(x.size(0), -1)
        z = self.layer(x)

        # z_hat = F.log_softmax(x, dim=-1)
        # z_hat.requires_grad = True

        loss = nn.functional.cross_entropy(z, y)

        self.log("training_loss", loss)

        return loss

    def forward(self, nx):
        # nx = self.transform(x).unsqueeze(0)
        # mnx = nx.view(nx.size(0), -1)
        # return self.layer(mnx)
        # nx = self.transform(x)
        # print("Shape of nx: ", nx.shape, nx.size())

        mnx = nx.reshape(nx.size(0), -1)
        # print("Shape of mnx: ", mnx.shape)
        return self.layer(mnx)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        # print("Gotten batch and shape of x: ", x.shape)

        x = x.view(x.size(0), -1)
        z = self.layer(x)

        loss = nn.functional.cross_entropy(z, y)

        self.log("training_loss", loss)


    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

cifar_module = CIFARTrainingModule(units=512)
data_module = CIFARDataModule()

trainer = pl.Trainer(max_epochs=config.epochs, accelerator='gpu', devices=1)

trainer.fit(cifar_module, data_module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | layer | Sequential | 1.7 M 
-------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.986     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/shivam13juna/Documents/virtual_envs/mlo/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:219: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/shivam13juna/Documents/virtual_envs/mlo/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:219: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/shivam13juna/Documents/virtual_envs/mlo/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1894: PossibleUserWarning: The number of training batches (5) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 19: 100%|██████████| 6/6 [00:06<00:00,  1.03s/it, loss=1.12, v_num=53]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 6/6 [00:06<00:00,  1.03s/it, loss=1.12, v_num=53]


In [4]:
cifar_module = cifar_module.eval()

In [5]:
transform = transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor(), transforms.Normalize(cifar_mean, cifar_std)])

tmp = datasets.CIFAR10('data/', train=False, download=True, transform=transform)

ano = iter(tmp)

x, y = next(ano)
# x = np.array(x)

Files already downloaded and verified


In [6]:
ind = list(range(10000))
# ind = [0, 0]

acc = 0
all = 0

for i in ind:
    inp = tmp[i][0]
    tar = tmp[i][1]
    out = np.argmax(cifar_module(inp.unsqueeze(0)).detach().numpy(), axis=1)

    if out == tar:
        acc += 1
    all += 1


print("Accuracy: ", acc/all)



Accuracy:  0.5209


6

In [30]:
# img = Image.open('tmp/0003.jpg')
img = Image.open('/home/shivam13juna/Documents/learn/tsai/emlo/s2/emlo-assn2/s7/10_pics/i7.jpg')
tmp.classes[np.argmax(cifar_module(transform(img).unsqueeze(0)).detach().numpy(), axis=1).tolist()[0]]

'truck'

In [ ]:

val = np.argmax(cifar_module(img).detach().numpy())
labels[val]